In [ ]:
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = t.device("cuda" if t.cuda.is_available() else "cpu")  # 设置使用GPU

In [ ]:
# 绘图

def draw(yi, color):
            plt.plot( np.arange(input.size(1)), yi[:input.size(1)], color, linewidth=2.0) # 画出已知数据的曲线
            plt.plot( np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth=2.0) # 画出预测的数据曲线 
def plot(y):
        plt.figure(figsize=(30, 10)) # 设置画布大小
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xlabel('x', fontsize=20) # x轴标签
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20) # x轴刻度
        plt.yticks(fontsize=20)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')

        plt.show()
        plt.close()

In [ ]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51) # 51是隐藏层的维度
        self.lstm2 = nn.LSTMCell(51, 51) # 51是隐藏层的维度
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future=0):
        outputs = []
        h_t = t.zeros(input.size(0), 51, dtype=t.double)
        c_t = t.zeros(input.size(0), 51, dtype=t.double)
        h_t2 = t.zeros(input.size(0), 51, dtype=t.double)
        c_t2 = t.zeros(input.size(0), 51, dtype=t.double)

        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = t.cat(outputs, dim=1)

        return outputs

In [ ]:
epochs = 10
# set random seed to 0
np.random.seed(0)
t.manual_seed(0)
# load data and make training set

data = t.load('traindata.pt')
input = t.from_numpy(data[3:, :-1]).to(device)
target = t.from_numpy(data[3:, 1:]).to(device)
test_input = t.from_numpy(data[:3, :-1]).to(device)
test_target = t.from_numpy(data[:3, 1:]).to(device)
# build the model
seq = Sequence() .to(device)
seq.double()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
 # begin to train
for i in range(epochs):
        print('STEP: ', i)

        def closure():
            optimizer.zero_grad()
            out = seq(input)
            loss = criterion(out, target)
            print('loss:', loss.item())
            loss.backward()
            return loss
        optimizer.step(closure)
        # begin to predict, no need to track gradient here
        with t.no_grad():
            future = 1000
            pred = seq(test_input, future=future)
            loss = criterion(pred[:, :-future], test_target)
            print('test loss:', loss.item())
            y = pred.detach().numpy()

            plot(y) # 画图